## parser.py

In [1]:
def parse_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

        total_clients = int(lines[0].strip())
        clients = []
        for i in range(1, total_clients+1):
            likes = set([w for w in lines[2*i-1].strip().split()[1:]])
            dislikes = set([w for w in lines[2*i].strip().split()[1:]])
            clients.append((likes, dislikes))
    return clients


## submit.py

In [2]:
def submit(filename, ingredients):
    with open(filename, 'w') as f:
        f.write(str(len(ingredients)) + ' ')
        f.write(' '.join(ingredients))
        

## algorithm.py

### Some intuition

- Files:
    - `a_an_example.in.txt` : Total ingredients: 6, total clients: 3
    - `b_basic.in.txt` : Total ingredients: 6, total clients: 5
    - `c_coarse.in.txt` : Total ingredients: 10, total clients: 10
    - `d_difficult.in.txt` : Total ingredients: 600, total clients: 9368
    - `e_elaborate.in.txt` : Total ingredients: 10000, total clients: 4986
- **Brute Force** is the first option by considering all possible sets of ingredients. It's possible for the first 3 files, but then it's not anymore feasible.
- **Greedy Search** As we can add any number of ingredients we want, non disliked ingredients should be part of the optimal solution
    - Thus we can build a solution by considering all the ingredients and then excluding the most disliked ones. Maybe local search could help.
    - One algorithm could be the following: 
        1. Add all the ingredients 
        2. Iteratively exclude the most disliked one if this leads to a better state.
    - Greedy search incrementally
        1. Add all the ingredients not disliked by anyone
        2. Iteratively add the most liked ingredient
    - The problem with the greedy search is that we do not consider specific subsets but all the elements have a specific order. e.g. if we have ingredients (a,b,c,d) with this specific liked/dislike order. The optimal solution might contain elements (a,d) which we would not be able to track down since greedy search would remove them one by one from left to right and the same applies to the incremental greedy search.
        - One way to tackle this problem, would be to use again greedy search, but instead looking all possible subsets for the next k elements.
        - Another way would be to do a branch and bound approach.
- **Branch and Bound approach**
    1. Initially the upper bound is the whole set of users
    2. For every ingredient we have two options. To add it in the pizza or not. 
        - If we add it in the pizza then the upper bound is reduced by the number of users that dislike it
        - If we do not add it in the pizza then the upper bound is reduced by the number of users that like it.
- **Graph Theoretic Approach**


In [16]:
import itertools

def get_file_specs(clients):
    
    dc = {}
    
    for c in clients:
        likes, dislikes = c

        for i in likes:
            if i not in dc:
                dc[i] = {'isliked': 1, 'isdisliked':0}
            else:
                dc[i]['isliked'] += 1

        for i in dislikes:
            if i not in dc:
                dc[i] = {'isliked': 0, 'isdisliked':1}
            else:
                dc[i]['isdisliked'] += 1
    
    # file specifications
    total_ingredients = len(dc.keys())
    total_clients = len(clients)    
    print(f'Total ingredients: {total_ingredients}, total clients: {total_clients}')
    
    dc = {k: v for k, v in sorted(dc.items(), key=lambda item: item[1]['isliked'],reverse=True)}
    ingredients = list(dc.keys())
    
    print('Most Liked Ingredients')
    for i in ingredients[0:10]:
        print(f'\t{i}: {dc[i]}')
        
    dc = {k: v for k, v in sorted(dc.items(), key=lambda item: item[1]['isdisliked'],reverse=True)}
    ingredients = list(dc.keys())
    print('Most Disliked Ingredients')
    for i in ingredients[0:10]:
        print(f'\t{i}: {dc[i]}')
        
    return 

def get_power_set(myset):
    all_combinations = itertools.chain(*[itertools.combinations(myset,i+1) for i,_ in enumerate(myset)])
    return list(all_combinations)

def evaluate_solution(clients, ingredients):
    """
    Takes as input the list of clients with their preferences and based on the selected ingredients
    returns the number of satisfied customers.
    """
    total_clients = 0
    
    for c in clients:
        likes, dislikes = c
        if likes.intersection(ingredients) == likes and not dislikes.intersection(ingredients):
            total_clients += 1
    
    return total_clients

def brute_force_algorithm(clients):
    """
    Produces all possible subsets and returns the best solution. 
    Could be used for instances up to rouhgly 30 ingredients
    """
    print('Applying brute force algorithm')
    ingredients = set()
    for c in clients:
        likes, dislikes = c
        ingredients = ingredients.union(likes.union(dislikes))
    power_set = get_power_set(ingredients)
    power_set.append(tuple())
    print(f'Total {len(power_set)} brute force cases.')
    max_clients, max_case = 0, set()
    for subset in power_set:
        total_clients = evaluate_solution(clients, subset)
        if total_clients > max_clients:
            max_clients = total_clients
            max_case = subset
            
    print(f'Set {max_case} maximizes total clients to {max_clients}')
    return max_case

def get_ingredient_prefetences(clients):
    dc = {}
    for c in clients:
        likes, dislikes = c

        for i in likes:
            if i not in dc:
                dc[i] = {'isliked': 1, 'isdisliked':0}
            else:
                dc[i]['isliked'] += 1

        for i in dislikes:
            if i not in dc:
                dc[i] = {'isliked': 0, 'isdisliked':1}
            else:
                dc[i]['isdisliked'] += 1
    return dc

def greedy_search(clients):
    """
    Add all possible ingredients and remove them one by one in decreasing order to how many people dislike them
    """
    print('Applying greedy search')

    dc = get_ingredient_prefetences(clients)
    dc = {k: v for k, v in sorted(dc.items(), key=lambda item: item[1]['isdisliked'],reverse=True)}
    disliked = list(dc.keys())

    ingredients = set(disliked)
        
    current_clients = evaluate_solution(clients, ingredients)
    best_solution = ingredients
    best_clients = current_clients
    i = 0
    while i < len(disliked):
        if i> 0 and i % 1000 == 0:
            print(i)
        ingredients = ingredients - set([disliked[i]])
        current_clients = evaluate_solution(clients, ingredients)
        if current_clients > best_clients:
            best_clients = current_clients
            best_solution = ingredients
        i = i + 1
    print(f'Greedy search found best solution: {best_clients} with {len(best_solution)} total ingredients')
    return best_solution

def incremental_greedy_search(clients):
    """
    Add all ingredients that are not disliked by anyone and add one by one all the ingredients
    in decreasing order of their likeness
    """
    print('Applying incremental greedy search')
    
    dc = get_ingredient_prefetences(clients)
    dc = {k: v for k, v in sorted(dc.items(), key=lambda item: item[1]['isliked'],reverse=True)}    
    liked = list(dc.keys())
    
    ingredients = set([k for k in dc.keys() if dc[k]['isdisliked'] == 0])
    print(f"Starting with {len(ingredients)} total ingredients")
    
    current_clients = evaluate_solution(clients, ingredients)
    best_solution = ingredients
    best_clients = current_clients
    i = 0
    while i < len(liked):
        if i> 0 and i % 1000 == 0:
            print(i)
        ingredients = ingredients.union(set([liked[i]]))
        current_clients = evaluate_solution(clients, ingredients)
        if current_clients > best_clients:
            best_clients = current_clients
            best_solution = ingredients
        i = i + 1
    print(f'Incremental greedy search found best solution: {best_clients} with {len(best_solution)} total ingredients')
    return best_solution


# test evaluation fuction
# clients = [({'peppers', 'cheese'}, set()), ({'basil'}, {'pineapple'}), ({'tomatoes', 'mushrooms'}, {'basil'})]
# ingredients = ['cheese', 'mushrooms', 'tomatoes', 'peppers']
# evaluate_solution(clients, ingredients)

# test powerset function
# myset = set([1,2,3])
# get_power_set(myset)

## run.py

In [17]:
import datetime
filenames = ['a_an_example.in.txt', 'b_basic.in.txt', 'c_coarse.in.txt', 'd_difficult.in.txt', 'e_elaborate.in.txt']
# for filename in filenames[:3]:
for filename in filenames[3:]:
    print('-'*100)
    print(filename)
    print('-'*100)
    input_path = 'input/' + filename
    output_path = 'output/' + filename
    # read data
    clients = parse_file(input_path)
    # run algorithm to get desired output
    get_file_specs(clients)
    
    print(f'Start of the greedy algorithm: {datetime.datetime.now()}')
    # ingredients = brute_force_algorithm(clients)
    # ingredients = greedy_search(clients)
    ingredients = incremental_greedy_search(clients)
    print(f'End of the greedy algorithm: {datetime.datetime.now()}')
    # store output in the proper file
    submit(output_path, ingredients)

----------------------------------------------------------------------------------------------------
d_difficult.in.txt
----------------------------------------------------------------------------------------------------
Total ingredients: 600, total clients: 9368
Most Liked Ingredients
	ingredient437: {'isliked': 59, 'isdisliked': 45}
	ingredient217: {'isliked': 56, 'isdisliked': 38}
	ingredient333: {'isliked': 54, 'isdisliked': 36}
	ingredient376: {'isliked': 54, 'isdisliked': 45}
	ingredient121: {'isliked': 52, 'isdisliked': 47}
	ingredient237: {'isliked': 52, 'isdisliked': 32}
	ingredient204: {'isliked': 52, 'isdisliked': 33}
	ingredient396: {'isliked': 51, 'isdisliked': 27}
	ingredient76: {'isliked': 51, 'isdisliked': 34}
	ingredient311: {'isliked': 50, 'isdisliked': 44}
Most Disliked Ingredients
	ingredient276: {'isliked': 26, 'isdisliked': 57}
	ingredient293: {'isliked': 48, 'isdisliked': 56}
	ingredient296: {'isliked': 30, 'isdisliked': 56}
	ingredient105: {'isliked': 46, 'isdi